<a href="https://colab.research.google.com/github/kdemertzis/EKPA/blob/main/Ransomware.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#εισαγωγή βιβλιοθηκών - πακέτων
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.covariance import EllipticEnvelope
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report, confusion_matrix
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import pandas as pd

In [ ]:
!pip install seaborn

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
#εισαγωγή του συνόλου δεδομένων
dataset=pd.read_csv("https://media.githubusercontent.com/media/kdemertzis/EKPA/main/Data/Ransomware.csv",sep='|')

In [ ]:
#eξερεύνηση συνόλου δεδομένων
dataset

In [ ]:
#περιγραφικά στατιστικά συνόλου δεδομένων
dataset.describe()

In [ ]:
# ανακάτεμα δεδομένων
dataset = dataset.sample(frac=1).reset_index(drop=True)
dataset.head()

In [ ]:
#tαξινόμηση δεδομένων με βάση το εάν είναι νόμιμο ή κακόβουλο λογισμικό
#0 legit, 1 ransomware
dataset.groupby(dataset['legitimate']).size()

In [ ]:
#γραφική απεικόνιση δεδομένων με βάση το εάν είναι νόμιμο ή κακόβουλο λογισμικό#
#0 legit, 1 ransomware
type_classify=['Malware', 'Legitimate']
count_classify=[41323,96724]
plt.pie(count_classify, labels=type_classify, autopct='%0.f%%')

In [ ]:
#οπτικοποίηση της κατανομής των δεδομένων
dataset.hist(bins = 50,figsize = (25,25))
plt.show()

In [ ]:
#πίνακας συσχέτισης χαρακτηριστικών του συνόλου δεδομένων
plt.figure(figsize=(15,13))
sns.heatmap(dataset.corr())
plt.show()

In [ ]:
#έλεγχος κενών τιμών στα δεδομένα
dataset.isnull().sum()

In [ ]:
# Ορισμός X και y
X = dataset.drop('legitimate', axis=1)
y = dataset['legitimate']

In [ ]:
# Χειρισμός τιμών που λείπουν με στρατηγική "συμπλήρωση με σταθερή τιμή"
imputer = SimpleImputer(strategy='constant', fill_value='unknown')
X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)
X_imputed

In [ ]:
# Ορίζουμε ένα LabelEncoder για κάθε στήλη με συμβολοσειρές
label_encoders = {}
for column in X_imputed.select_dtypes(include='object').columns:
    label_encoders[column] = LabelEncoder()
    X_imputed[column] = label_encoders[column].fit_transform(X_imputed[column])

In [ ]:
# Δημιουργία του OneClassSVM με προεπιλεγμένες ρυθμίσεις για τον εντοπισμό ακραίων τιμών
outlier_detector = OneClassSVM(nu=0.05)
outliers = outlier_detector.fit_predict(X_imputed)

In [ ]:
# Επαναφορά των αρχικών τιμών συμβολοσειρών αν χρειαστεί
for column in X_imputed.select_dtypes(include='object').columns:
    X_imputed[column] = label_encoders[column].inverse_transform(X_imputed[column])

In [ ]:
# Εκτύπωση των ακραίων τιμών
print("Ακραίες Τιμές:\n", outliers)

In [ ]:
# Δημιουργία νέου συνόλου δεδομένων μετά την αφαίρεση των ακραίων τιμών
clean_data = pd.DataFrame(X_imputed[outliers != -1])
clean_data['legitimate'] = y[outliers != -1]

In [ ]:
# Αναφορά του νέου συνόλου ανά κλάση
remaining_instances_report = clean_data['legitimate'].value_counts()
print("Αναφορά υπολοίπων περιπτώσεων ανά κατηγορία:\n", remaining_instances_report)

In [ ]:
# Εκτύπωση καθαρού συνόλου
clean_data

In [ ]:
pip install -U ppscore

In [ ]:

import ppscore as pps

In [ ]:
# Χρήση μόνο των αριθμητικών στηλών από το clean_data
numeric_clean = clean_data.select_dtypes(include='number')

In [ ]:
print("Αριθμητικές στήλες που θα χρησιμοποιηθούν για PPS:", numeric_clean.columns.tolist())

In [ ]:
# Υπολογισμός πίνακα PPS μεταξύ των αριθμητικών στηλών
pps_matrix = pps.matrix(numeric_clean)

In [ ]:
# Έλεγχος μερικών γραμμών για επιβεβαίωση
print(pps_matrix.head())

In [ ]:
# Οπτικοποίηση heatmap πίνακα PPS
plt.figure(figsize=(12, 10))
sns.heatmap(
    pps_matrix.pivot(columns='x', index='y', values='ppscore'),
    vmin=0, vmax=1, cmap='Blues', linewidths=0.5, annot=True
)
plt.title('PPS Matrix μεταξύ αριθμητικών μεταβλητών')
plt.show()

In [32]:
predictors_for_legit = pps.predictors(numeric_clean, y='legitimate')
print("Predictors for 'legitimate' (first rows):")
print(predictors_for_legit.head(10))

Predictors for 'legitimate' (first rows):
                        x           y   ppscore        case  is_valid_score  \
0                    Name  legitimate  1.000000  regression            True   
1         Characteristics  legitimate  0.781023  regression            True   
2     ResourcesMaxEntropy  legitimate  0.741983  regression            True   
3        ResourcesMaxSize  legitimate  0.734803  regression            True   
4     ResourcesMinEntropy  legitimate  0.731894  regression            True   
5               ImageBase  legitimate  0.709898  regression            True   
6      SectionsMinEntropy  legitimate  0.649989  regression            True   
7  SectionsMinVirtualsize  legitimate  0.639043  regression            True   
8      SectionsMaxEntropy  legitimate  0.635500  regression            True   
9  VersionInformationSize  legitimate  0.631424  regression            True   

                metric  baseline_score  model_score                    model  
0  mean a